# Lookout Mobile Malware Threat Hunting

This notebook provides threat hunting queries for investigating mobile malware detected by Lookout Mobile Risk API.

## Prerequisites
- Lookout data connector configured in Microsoft Sentinel
- LookoutEvents parser deployed
- Access to Log Analytics workspace

## 1. High Severity Malware Detections

Identify devices with critical or high severity malware threats.

In [1]:
LookoutEvents
| where EventType == "THREAT"
| where ThreatSeverity in ("CRITICAL", "HIGH")
| where ThreatType in ("MALWARE", "APPLICATION")
| summarize 
    ThreatCount = count(),
    Applications = make_set(ThreatApplicationName),
    PackageHashes = make_set(ThreatPackageSha)
    by DeviceEmailAddress, DevicePlatform, ThreatSeverity
| sort by ThreatCount desc

DeviceEmailAddress,DevicePlatform,ThreatSeverity,ThreatCount,Applications,PackageHashes
john.smith@contoso.com,ANDROID,CRITICAL,5,"[""FakeBank.apk"", ""SpyAgent.apk""]","[""a1b2c3d4e5f6..."", ""f6e5d4c3b2a1...""]"
sarah.jones@contoso.com,ANDROID,HIGH,3,"[""MalwareApp.apk""]","[""1234abcd5678...""]"
mike.wilson@contoso.com,IOS,HIGH,2,"[""SuspiciousApp""]","[""9876fedc5432...""]"
lisa.chen@contoso.com,ANDROID,CRITICAL,2,"[""TrojanSMS.apk""]","[""abcdef123456...""]"
david.brown@contoso.com,ANDROID,HIGH,1,"[""RiskApp.apk""]","[""567890abcdef...""]"


## 2. Malware Package Hash Analysis

Analyze malicious package hashes across the fleet to identify widespread campaigns.

In [2]:
LookoutEvents
| where EventType == "THREAT"
| where isnotempty(ThreatPackageSha)
| summarize 
    DevicesAffected = dcount(DeviceEmailAddress),
    FirstSeen = min(TimeGenerated),
    LastSeen = max(TimeGenerated),
    AppNames = make_set(ThreatApplicationName),
    Severities = make_set(ThreatSeverity)
    by ThreatPackageSha
| where DevicesAffected > 1
| sort by DevicesAffected desc

ThreatPackageSha,DevicesAffected,FirstSeen,LastSeen,AppNames,Severities
a1b2c3d4e5f6g7h8i9j0...,12,2025-01-01T08:15:00Z,2025-01-05T14:30:00Z,"[""FakeBank.apk""]","[""CRITICAL""]"
f6e5d4c3b2a1z9y8x7w6...,8,2025-01-02T10:22:00Z,2025-01-05T11:45:00Z,"[""SpyAgent.apk"", ""DataStealer.apk""]","[""CRITICAL"", ""HIGH""]"
1234abcd5678efgh9012...,5,2025-01-03T09:00:00Z,2025-01-04T16:20:00Z,"[""MalwareApp.apk""]","[""HIGH""]"
9876fedc5432abcd1098...,3,2025-01-04T12:00:00Z,2025-01-05T08:00:00Z,"[""TrojanSMS.apk""]","[""HIGH""]"


## 3. Malware Timeline Analysis

Track malware detection trends over time to identify attack campaigns.

In [3]:
LookoutEvents
| where EventType == "THREAT"
| where ThreatType in ("MALWARE", "APPLICATION")
| summarize Count = count() by bin(TimeGenerated, 1h), ThreatSeverity
| render timechart

Time Chart rendered - Malware detection trends showing CRITICAL and HIGH severity threats over the past week

## 4. Unresolved Malware Threats

Identify devices with malware that hasn't been remediated.

In [4]:
LookoutEvents
| where EventType == "THREAT"
| where ThreatType in ("MALWARE", "APPLICATION")
| where ThreatStatus == "OPEN"
| project 
    TimeGenerated,
    DeviceEmailAddress,
    DevicePlatform,
    ThreatApplicationName,
    ThreatSeverity,
    ThreatDescription,
    ThreatPackageSha
| sort by TimeGenerated desc

TimeGenerated,DeviceEmailAddress,DevicePlatform,ThreatApplicationName,ThreatSeverity,ThreatDescription,ThreatPackageSha
2025-01-05T14:30:00Z,john.smith@contoso.com,ANDROID,FakeBank.apk,CRITICAL,Banking trojan that steals credentials,a1b2c3d4e5f6...
2025-01-05T12:15:00Z,sarah.jones@contoso.com,ANDROID,SpyAgent.apk,CRITICAL,Spyware collecting user data,f6e5d4c3b2a1...
2025-01-05T10:45:00Z,mike.wilson@contoso.com,IOS,SuspiciousApp,HIGH,Potentially unwanted application,9876fedc5432...
2025-01-04T16:20:00Z,lisa.chen@contoso.com,ANDROID,TrojanSMS.apk,HIGH,SMS premium service fraud,abcdef123456...
2025-01-04T09:00:00Z,david.brown@contoso.com,ANDROID,RiskApp.apk,HIGH,Risky app with privacy concerns,567890abcdef...
